# Reading JSON input file and extracting data

In [1]:
import json   
import pandas as pd
import traceback

In [2]:
f1 = open("Sample_Data.json" , 'r')

lines = f1.readlines()

tweets = {}
tweets['tweets'] = []

for line in lines:
    if line[0] == '{':
        tweets['tweets'].append(json.loads(line)) 

In [3]:
tweet = tweets['tweets']
#tweet[0]
#tweet[0]['retweeted_status']['user']

In [4]:
#df_twitter = pd.DataFrame(tweet)

In [5]:
#df_twitter.head(2)

In [6]:
#df_twitter_user = df_twitter[['user']]
#df_twitter_user.head(5)

In [7]:
tweet[0]['user']['id_str']

'1242817830946508801'

In [8]:
len(tweet)

2001

## Connection to postgresql database

In [83]:
import psycopg2 # psycopg2 is a PostgreSQL database adapter for the Python , and it provides a Python interface for connecting to and interacting with PostgreSQL databases.

In [84]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="TwitterDatabase",
    user="postgres",
    password="India@2194"
)


In [85]:
# Create a cursor object to execute PostgreSQL commands
cursor = conn.cursor()

In [86]:
#cursor.execute("CREATE DATABASE TestDB")
#conn.commit()

## Create users table in postgres

In [72]:
#Create users table

cursor.execute("""
    CREATE TABLE users (
        id_str VARCHAR(50) PRIMARY KEY UNIQUE,
        name VARCHAR(500),
        screen_name VARCHAR(500),
        description VARCHAR(5000),
        verified BOOLEAN,
        followers_count INT,
        friends_count INT,
        favourites_count INT,
        statuses_count INT,
        created_at TIMESTAMP,
        protected BOOLEAN
    )
""")


conn.commit()

In [87]:
#datetime conversion
def convert_timestamp(created_at):
    timestamp_obj = datetime.strptime(created_at, '%a %b %d %H:%M:%S %z %Y')
    created_at = datetime.utcfromtimestamp(timestamp_obj.replace(tzinfo=timezone.utc).timestamp())
    return created_at

In [88]:
import datetime
# Get the current time as a datetime object
now = datetime.datetime.now()

In [89]:
tweet_user_list = list()

## Insert records in users table

In [91]:
# insert query with values to be inserted
print("Started inserting rows at time:",now.time())

print("Total rows of data available: ",len(tweet))

inserted_rows_count = 0
Skipped_rows_count = 0

user_count = len(tweet)

for i in range(0,len(tweet)):
    try:
        if (tweet[i]['user']['id_str']) not in tweet_user_list:            
            
            if  user_count > 0 :
                
            # Inserting user data user field
                cursor.execute("INSERT INTO users (id_str,name,screen_name,description,verified,followers_count,friends_count ,favourites_count,statuses_count,created_at ,protected) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (tweet[i]['user']['id_str'],tweet[i]['user']['name'],tweet[i]['user']['screen_name'],tweet[i]['user']['description'],tweet[i]['user']['verified'],tweet[i]['user']['followers_count'],tweet[i]['user']['friends_count'],tweet[i]['user']['favourites_count'],tweet[i]['user']['statuses_count'],tweet[i]['user']['created_at'],tweet[i]['user']['protected'] ))
                #print('Record inserted1 ----------->')
                tweet_user_list.append(tweet[i]['user']['id_str'])
                inserted_rows_count = inserted_rows_count + 1
            
            # Inserting user data from Retweeted status field
            else:                
                
                cursor.execute("INSERT INTO users (id_str,name,screen_name,description,verified,followers_count,friends_count,favourites_count,statuses_count,created_at ,protected) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (tweet[i]['retweeted_status']['user']['id_str'],tweet[i]['retweeted_status']['user']['name'],tweet[i]['retweeted_status']['user']['screen_name'],tweet[i]['retweeted_status']['user']['description'],tweet[i]['user']['verified'],tweet[i]['retweeted_status']['user']['followers_count'],tweet[i]['retweeted_status']['user']['friends_count'],tweet[i]['retweeted_status']['user']['favourites_count'],tweet[i]['retweeted_status']['user']['statuses_count'],tweet[i]['retweeted_status']['user']['created_at'],tweet[i]['retweeted_status']['user']['protected'] ))
                #print('Record inserted2 ---------------->')
                tweet_user_list.append(tweet[i]['retweeted_status']['user']['id_str'])
                inserted_rows_count = inserted_rows_count + 1
                      
        else:
            
            Skipped_rows_count = Skipped_rows_count + 1
            continue
            
    except Exception as e:
        print("Error encountered:",e)
        print("Trace:")
        traceback.print_exc()
        conn.commit()
        # Roll back the transaction
        conn.rollback()
        continue

conn.commit()

print("Inserted_rows_count: ",inserted_rows_count)
print("Skipped_rows_count: ",Skipped_rows_count)
print("Finished inserting rows at time:",now.time())


Started inserting rows at time: 18:12:04.746159
Total rows of data available:  2001
Inserted_rows_count:  1851
Skipped_rows_count:  150
Finished inserting rows at time: 18:12:04.746159
